In [1]:
import pandas as pd
pd.set_option('float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 999)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    "../Datasets/LoanStats3d.csv",
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,nan,24700.00,24700.00,24700.00,36 months,11.99%,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.00,Not Verified,Dec-2015,Fully Paid,n,nan,NaN,small_business,Business,577xx,SD,16.06,1.00,Dec-1999,4.00,6.00,nan,22.00,0.00,21470.00,19.2%,38.00,w,0.00,0.00,25679.66,25679.66,24700.00,979.66,0.00,0.00,0.00,Jun-2016,926.35,NaN,Apr-2019,0.00,nan,1.00,Individual,nan,nan,NaN,0.00,0.00,204396.00,1.00,1.00,0.00,1.00,19.00,18005.00,73.00,2.00,3.00,6472.00,29.00,111800.00,0.00,0.00,6.00,4.00,9733.00,57830.00,27.10,0.00,0.00,113.00,192.00,2.00,2.00,4.00,2.00,nan,0.00,6.00,0.00,5.00,5.00,13.00,17.00,6.00,20.00,27.00,5.00,22.00,0.00,0.00,0.00,2.00,97.40,7.70,0.00,0.00,314017.00,39475.00,79300.00,24667.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,nan
1,NaN,nan,35000.00,35000.00,35000.00,60 months,14.85%,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.00,Source Verified,Dec-2015,Current,n,nan,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.00,Sep-2008,0.00,nan,nan,13.00,0.00,7802.00,11.6%,17.00,w,15264.49,15264.49,32293.91,32293.91,19735.51,12558.40,0.00,0.00,0.00,Apr-2019,829.90,Apr-2019,Apr-2019,0.00,nan,1.00,Individual,nan,nan,NaN,0.00,0.00,301500.00,1.00,1.00,0.00,1.00,23.00,12609.00,70.00,1.00,1.00,6987.00,45.00,67300.00,0.00,1.00,0.00,2.00,23192.00,54962.00,12.10,0.00,0.00,36.00,87.00,2.00,2.00,1.00,2.00,nan,nan,nan,0.00,4.00,5.00,8.00,10.00,2.00,10.00,13.00,5.00,13.00,0.00,0.00,0.00,1.00,100.00,0.00,0.00,0.00,381215.00,52226.00,62500.00,18000.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,NaN,nan,nan,nan,N,NaN,NaN,NaN,nan,nan,nan
2,NaN,nan,3600.

In [4]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [6]:
y2015.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.00,0.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421093.00,421095.00,421095.00,217133.00,74415.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,122729.00,421095.00,511.00,509.00,421095.00,421095.00,421095.00,21372.00,21372.00,21372.00,21372.00,20810.00,21372.00,18617.00,21372.00,21372.00,21372.00,21372.00,421095.00,21372.00,21372.00,21372.00,421095.00,421095.00,417132.00,416868.00,421095.00,421095.00,408841.00,421095.00,421095.00,421095.00,421095.00,417297.00,108600.00,376496.00,151737.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,421094.00,421095.00,421095.00,401865.00,421095.00,421095.00,421095.00,421095.00,416856.00,421095.00,421095.00,421095.00,421095.00,421095.00,421095.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2590.00,2590.00,2590.00,2590.00,2106.00,2590.00,2590.00,11695.00,11695.00,11695.00
mean,nan,nan,15240.29,15240.29,15234.16,12.60,441.84,76965.61,19.15,0.35,0.57,34.02,66.59,11.95,0.23,17685.48,25.49,706.47,706.12,15835.52,15829.30,12586.60,3010.35,1.87,236.70,42.25,4291.71,0.02,43.81,1.00,109981.01,18.31,0.01,257.48,140075.55,1.11,2.93,0.76,1.67,20.91,36552.81,71.58,1.39,2.98,5887.98,60.88,33657.71,0.94,1.54,2.23,4.66,13104.78,9636.06,62.23,0.01,13.77,127.57,186.59,13.45,8.01,1.69,24.86,39.84,6.75,35.80,0.53,3.74,5.86,4.80,8.14,8.65,8.43,14.88,5.81,11.89,0.00,0.00,0.09,2.17,93.92,47.53,0.13,0.06,173407.01,52221.99,21958.21,43478.65,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3.00,118.30,3.00,14.22,350.53,9278.74,183.12,4853.92,47.19,12.83
std,nan,nan,8571.33,8571.33,8567.94,4.32,244.85,73949.96,8.89,0.93,0.87,21.99,25.58,5.63,0.65,24195.03,12.10,2270.10,2268.88,10073.05,10069.17,8415.21,3003.82,12.97,971.75,174.35,6358.81,0.16,21.32,0.00,52730.38,7.17,0.09,2240.65,155269.60,1.24,3.09,1.00,1.69,27.21,43103.83,23.02,1.52,2.63,5284.70,20.01,37664.04,1.45,2.72,2.48,3.20,15671.80,14774.17,27.58,0.12,781.82,52.08,95.37,17.16,9.19,1.99,32.27,23.00,5.91,22.63,1.33,2.30,3.44,3.04,4.82,7.47,4.66,8.29,3.35,5.62,0.03,0.07,0.52,1.85,8.88,36.05,0.39,0.44,175672.22,49074.94,21840.37,43942.71,nan,nan,nan,nan,nan,nan,nan,n

## Reference model

In [7]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

score = cross_val_score(rfc, X, Y, cv=10)
print("Accuracy: %0.3f (+/- %0.3f)" % (score.mean(), score.std() * 2))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

Accuracy: 0.992 (+/- 0.003)


## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [10]:
col_y = y2015['loan_status']
y2015.drop(labels=['loan_status'], axis=1,inplace = True)
y2015.insert(0, 'loan_status', col_y)
df_dumm = pd.get_dummies(y2015)
df_dumm = df_dumm.dropna(axis=1)

In [11]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=10):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df_dumm, 100).to_string())

Top Absolute Correlations
loan_amnt                                      funded_amnt                              1.00
hardship_start_date_Apr-2017                   payment_plan_start_date_Apr-2017         1.00
hardship_flag_Y                                hardship_status_ACTIVE                   1.00
hardship_flag_N                                hardship_status_ACTIVE                   1.00
purpose_educational                            title_Learning and training              1.00
hardship_end_date_Jul-2019                     payment_plan_start_date_May-2019         1.00
hardship_flag_N                                hardship_flag_Y                          1.00
application_type_Joint App                     verification_status_joint_Not Verified   1.00
application_type_Individual                    verification_status_joint_Not Verified   1.00
hardship_end_date_Jun-2019                     payment_plan_start_date_Apr-2019         1.00
application_type_Individual                 

In [12]:
# Your code here.

rfc = ensemble.RandomForestClassifier()
X = y2015.drop([ 
               "loan_status", # is Y 
               "id",
               "member_id",
               "loan_amnt",
               "funded_amnt",
               "funded_amnt_inv",
               "term",
               "int_rate",
               "installment",
               "grade",
               "emp_length",
               "home_ownership",
               "annual_inc",
               "verification_status",
               "issue_d",
               "pymnt_plan",
               "purpose",
               "title",
               "dti",
               "delinq_2yrs",
               "inq_last_6mths",
               "mths_since_last_delinq",
               "mths_since_last_record",
               "open_acc",
               "pub_rec",
               "revol_bal",
               "total_acc",
               "initial_list_status",
               #"out_prncp",
               #"out_prncp_inv",
               "total_pymnt",
               "total_pymnt_inv",
               "total_rec_prncp",
               "total_rec_int",
               "total_rec_late_fee",
               #"recoveries",
               "collection_recovery_fee",
               "last_pymnt_d",
               "last_pymnt_amnt",
               "next_pymnt_d",
               "last_credit_pull_d",
               "collections_12_mths_ex_med",
               "mths_since_last_major_derog",
               "policy_code",
               "application_type",
               "annual_inc_joint",
               "dti_joint",
               "verification_status_joint",
               "acc_now_delinq",
               "tot_coll_amt",
               "tot_cur_bal",
               "open_acc_6m",
               "open_act_il",
               "open_il_12m",
               "open_il_24m",
               "mths_since_rcnt_il",
               "total_bal_il",
               "il_util",
               "open_rv_12m",
               "open_rv_24m",
               "max_bal_bc",
               "all_util",
               "total_rev_hi_lim",
               "inq_fi",
               "total_cu_tl",
               "inq_last_12m",
               "acc_open_past_24mths",
               "avg_cur_bal","bc_open_to_buy",
               "bc_util",
               "chargeoff_within_12_mths",
               "delinq_amnt",
               "mo_sin_old_il_acct",
               "mo_sin_old_rev_tl_op",
               "mo_sin_rcnt_rev_tl_op",
               "mo_sin_rcnt_tl",
               "mort_acc",
               "mths_since_recent_bc",
               "mths_since_recent_bc_dlq",
               "mths_since_recent_inq",
               "mths_since_recent_revol_delinq",
               "num_accts_ever_120_pd",
               "num_actv_bc_tl","num_actv_rev_tl",
               "num_bc_sats",
               "num_bc_tl","num_il_tl",
               "num_op_rev_tl",
               "num_rev_accts",
               "num_rev_tl_bal_gt_0",
               "num_sats",
               "num_tl_120dpd_2m",
               "num_tl_30dpd",
               "num_tl_90g_dpd_24m",
               "num_tl_op_past_12m",
               "pct_tl_nvr_dlq",
               "percent_bc_gt_75","pub_rec_bankruptcies",
               "tax_liens","tot_hi_cred_lim",
               "total_bal_ex_mort",
               "total_bc_limit",
               "total_il_high_credit_limit",
               "revol_bal_joint",
               "sec_app_earliest_cr_line",
               "sec_app_inq_last_6mths",
               "sec_app_mort_acc","sec_app_open_acc",
               "sec_app_revol_util","sec_app_open_act_il",
               "sec_app_num_rev_accts",
               "sec_app_chargeoff_within_12_mths",
               "sec_app_collections_12_mths_ex_med",
               "sec_app_mths_since_last_major_derog",
               "hardship_flag",
               "hardship_type",
               "hardship_reason",
               "hardship_status",
               "deferral_term",
               "hardship_amount",
               "hardship_start_date",
               "hardship_end_date",
               "payment_plan_start_date",
               "hardship_length",
               "hardship_dpd",
               "hardship_loan_status",
               "orig_projected_additional_accrued_interest",
               "hardship_payoff_balance_amount",
               "hardship_last_payment_amount",
               "debt_settlement_flag",
               "debt_settlement_flag_date",
               "settlement_status",
               "settlement_date",
               "settlement_amount",
               "settlement_percentage",
               "settlement_term" ], 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

score = cross_val_score(rfc, X, Y, cv=10)
print("Accuracy: %0.3f (+/- %0.3f)" % (score.mean(), score.std() * 2))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

Accuracy: 0.941 (+/- 0.007)


### Drill achieved : Accuracy: 0.940 (+/- 0.007)